In [ ]:
#Project pipeline
# Text cleaning ----- ??
# Coreference resolution  ------ To be done?
# Named entity recognition ---- How to extract the missing ones?

# Entity linking ----
# Relation extraction

In [2]:
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import reuters
import re
import spacy
nlp = spacy.load('en_core_web_lg')#, disable=['parser', 'tagger'])
reuters_fileids_crude = reuters.fileids(categories=['crude'])



# reuters_nlp = [nlp(re.sub('\s+',' ', reuters.raw(i).strip())) for i in reuters_fileids[:100]]
# reuters_nlp_crude = [nlp(re.sub('\s+',' ', reuters.raw(i).strip())) for i in reuters_fileids_crude]
# reuters_nlp_crude
# length = [[i,  reuters.raw(i).strip()] for i in reuters_fileids_crude]
# label_counter = Counter()
# reuters.words()
# reuters.sents()
# doc = reuters_nlp[50]

In [2]:
reuters_text = [reuters.raw(i).strip() for i in reuters_fileids_crude]
text = reuters_text[25].replace("\n", " ").strip()
doc = nlp(text)

In [41]:
# doc = reuters_nlp_crude[25]
# len(doc)
# doc
# [len(reuters_nlp[i]) for i in reuters_fileids(categories=['fuel'])] #reuters_fileids(categories=['fuel']

# len(reuters.paras())
# reuters.paras()[5]

ABU DHABI MARKETING SAID NOT BREACHING OPEC PACT   A senior Abu Dhabi oil official said   in remarks published today the emirate, largest producer in the   United Arab Emirates (UAE), was succeeding in marketing its   crude oil without breaching OPEC accords.       Khalaf al-Oteiba, Marketing Director at the Abu Dhabi   National Oil Co (ADNOC), told the company's Petroleum Community   magazine ADNOC was also keen to keep good customer relations.   "The company will maintain its dialogue with and care for its   customers in accordance with market conditions...And take   necessary steps to guarantee marketing its production," he said.      "The present oil marketing policy of ADNOC is based on   adherence to OPEC decisions of December 1986 to control   production and establish a new pricing system in an attempt to   stabilize the market," he added.       OPEC agreed last December to limit production to 15.8 mln   bpd and return to fixed prices averaging 18 dlrs a barrel.       Oteiba sai

In [4]:
print(*nlp.pipeline, sep='\n')

('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7fc56dee8de0>)
('tagger', <spacy.pipeline.tagger.Tagger object at 0x7fc56dee8d00>)
('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7fc56dc249d0>)
('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7fc56db746e0>)
('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x7fc56db82c30>)
('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7fc56dc24b50>)


In [8]:
import pandas as pd
list_ent = []
for i in range(len(doc)):
    for ent in doc[i].ents:
# for ent in doc.ents:
    #check if entity is equal 'LOC' or 'GPE'
        if ent.label_ in ['LOC', 'GPE', 'ORG']:
            if ent.text not in list_ent:
                list_ent.append([ent.text, ent.label_])
# list_ent
df = pd.DataFrame(list_ent, columns=["entity", "type"])
# df.groupby('type').value_counts()
df['type'].value_counts()
df.head(10)


AttributeError: 'spacy.tokens.token.Token' object has no attribute 'ents'

In [13]:
df.to_csv('data/list_ent.csv')

In [13]:
def reset_pipeline(nlp, pipes):
    # remove all custom pipes
    custom_pipes = [pipe for (pipe, _) in nlp.pipeline
                    if pipe not in ['tagger', 'parser', 'ner',
                                    'tok2vec', 'attribute_ruler', 'lemmatizer']]
    for pipe in custom_pipes:
        _ = nlp.remove_pipe(pipe)
    # re-add specified pipes
    for pipe in pipes:
        if 'neuralcoref' == pipe or 'neuralcoref' in str(pipe.__class__):
            nlp.add_pipe(pipe, name='neural_coref')
        else:
            nlp.add_pipe(pipe)

    print(f"Model: {nlp.meta['name']}, Language: {nlp.meta['lang']}")
    print(*nlp.pipeline, sep='\n')
    
reset_pipeline(nlp, ['init_coref'])

ValueError: [E002] Can't find factory for 'init_coref' for language English (en). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a Transformer, make sure to install 'spacy-transformers'. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, entity_linker, ner, beam_ner, entity_ruler, lemmatizer, tagger, morphologizer, senter, sentencizer, textcat, spancat, textcat_multilabel, en.lemmatizer

## Relationship extraction (Vanessa)

In [ ]:
from spacy.tokens import Token
from spacy import Language

Token.set_extension('ref_n', default='', force = True)
Token.set_extension('ref_t', default='', force = True)

@Language.component("init_coref")
def init_coref(doc):
    for e in doc.ents:
        if e.label_ in ['ORG', 'GOV', 'PERSON']:
            e[0]._.ref_n, e[0]._.ref_t = e.text, e.label_
    return doc

def reset_pipeline(nlp, pipes):
    # remove all custom pipes
    custom_pipes = [pipe for (pipe, _) in nlp.pipeline
                    if pipe not in ['tagger', 'parser', 'ner',
                                    'tok2vec', 'attribute_ruler', 'lemmatizer']]
    for pipe in custom_pipes:
        _ = nlp.remove_pipe(pipe)
    # re-add specified pipes
    for pipe in pipes:
        if 'neuralcoref' == pipe or 'neuralcoref' in str(pipe.__class__):
            nlp.add_pipe(pipe, name='neural_coref')
        else:
            nlp.add_pipe(pipe)

    print(f"Model: {nlp.meta['name']}, Language: {nlp.meta['lang']}")
    print(*nlp.pipeline, sep='\n')
    
reset_pipeline(nlp, ['init_coref'])

## Using dependency trees (From Vanessa)

In [ ]:
options = {"bg": "white", "distance": 130,
           "color": "black", "font": "Source Sans Pro"}
for sent in doc.sents:
    displacy.render(sent, style="dep", options=options)
    # displacy.render(doc, style="dep", options=options)

In [ ]:
# Actually we search for the shortest path between the
# subject running through our predicate (verb) to the object.
# subject and object are organizations in our examples.

# Here are the three helper functions omitted in the book:
# - bfs: breadth first searching the closest subject/object 
# - is_passive: checks if noun or verb is in passive form
# - find_subj: searches left part of tree for subject
# - find_obj: searches right part of tree for object

from collections import deque

def bfs(root, ent_type, deps, first_dep_only=False):
    """Return first child of root (included) that matches
    ent_type and dependency list by breadth first search.
    Search stops after first dependency match if first_dep_only
    (used for subject search - do not "jump" over subjects)"""
    to_visit = deque([root]) # queue for bfs

    while len(to_visit) > 0:
        child = to_visit.popleft()
        # print("child", child, child.dep_)
        if child.dep_ in deps:
            if child._.ref_t == ent_type:
                return child
            elif first_dep_only: # first match (subjects)
                return None
        elif child.dep_ == 'compound' and \
             child.head.dep_ in deps and \
             child._.ref_t == ent_type: # check if contained in compound
            return child
        to_visit.extend(list(child.children))
    return None

def is_passive(token):
    if token.dep_.endswith('pass'): # noun
        return True
    for left in token.lefts: # verb
        if left.dep_ == 'auxpass':
            return True
    return False

def find_subj(pred, ent_type, passive):
    """Find closest subject in predicates left subtree or
    predicates parent's left subtree (recursive).
    Has a filter on organizations."""
    for left in pred.lefts:
        if passive: # if pred is passive, search for passive subject
            subj = bfs(left, ent_type, ['nsubjpass', 'nsubj:pass'], True)
        else:
            subj = bfs(left, ent_type, ['nsubj'], True)
        if subj is not None: # found it!
            return subj
    if pred.head != pred and not is_passive(pred): 
        return find_subj(pred.head, ent_type, passive) # climb up left subtree
    else:
        return None

def find_obj(pred, ent_type, excl_prepos):
    """Find closest object in predicates right subtree.
    Skip prepositional objects if the preposition is in exclude list.
    Has a filter on organizations."""
    for right in pred.rights:
        obj = bfs(right, ent_type, ['dobj', 'pobj', 'iobj', 'obj', 'obl'])
        if obj is not None:
            if obj.dep_ == 'pobj' and obj.head.lemma_.lower() in excl_prepos: # check preposition
                continue
            return obj
    return None

def extract_rel_dep(doc, pred_name, pred_synonyms, excl_prepos=[]):
    for token in doc:
        if token.pos_ == 'VERB' and token.lemma_ in pred_synonyms:
            pred = token
            passive = is_passive(pred)
            subj = find_subj(pred, 'ORG', passive)
            if subj is not None:
                obj = find_obj(pred, 'ORG', excl_prepos)
                if obj is not None:
                    if passive: # switch roles
                        obj, subj = subj, obj
                    yield ((subj._.ref_n, subj._.ref_t), pred_name, 
                           (obj._.ref_n, obj._.ref_t))

In [ ]:
# text = """Fujitsu said that Schlumberger Ltd has arranged 
# to buy its stake in Fairchild Inc."""
text = """Iraq said today its troops were pushing Iranian forces out of positions they had initially 
occupied when they launched a new offensive near the southern port of Basra early yesterday."""
doc = nlp(text)
print(*extract_rel_dep(doc, 'push', ['pushing']), sep='\n')

In [ ]:
print(*extract_rel_dep(doc, 'sell', ['sells']), sep='\n')

## Collects Chunks from each sentence

In [30]:
chunks_list = []
# for i in range(len(doc)):
for sent in doc.sents:
    for chunk in doc.noun_chunks:
        if chunk.text not in chunks_list:
            chunks_list.append([chunk.text, chunk.root.text , chunk.root.dep_,
            chunk.root.head.text])
            # print(chunk.text +'---', chunk.root.text +'---', chunk.root.dep_,
            #     chunk.root.head.text)

In [31]:
# chunks_list to csv
df = pd.DataFrame(chunks_list, columns=["text", "root Text", "root dep", "head text"])
# df.groupby('type').value_counts()
# df['type'].value_counts()
df.head(10)
df.to_csv('data/chunks_list.csv')

## Coreference

In [4]:
import spacy
import neuralcoref
from spacy import displacy

neuralcoref.add_to_pipe(nlp)

/home/mokegg/anaconda3/envs/tf/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/home/mokegg/anaconda3/envs/tf/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
/home/mokegg/anaconda3/envs/tf/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)


In [5]:
doc2 = nlp(doc._.coref_resolved)
displacy.render(doc, style="ent")
print('*****' '\n\n')
displacy.render(doc2, style="ent")

*****




/home/mokegg/anaconda3/envs/tf/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


### Jaal

In [4]:
from jaal import Jaal
from jaal.datasets import load_got
import pandas as pd

#load the data

edge_df, node_df = load_got()

# add edge labels
edge_df.loc[:, 'label'] = edge_df.loc[:, 'weight'].astype(str)

edge_df.head(), node_df.head()

(                              from                  to  weight strength label
 0  Aemon-Targaryen-(Maester-Aemon)        Jeor-Mormont      13   medium    13
 1  Aemon-Targaryen-(Maester-Aemon)            Jon-Snow      34   medium    34
 2               Aerys-II-Targaryen    Robert-Baratheon      12   medium    12
 3                             Aggo  Daenerys-Targaryen      11   medium    11
 4                   Alliser-Thorne            Jon-Snow      32   medium    32,
                   id  gender  screentime
 0    Illyrio-Mopatis    male        3.30
 1        Jory-Cassel    male        6.15
 2  Viserys-Targaryen    male       20.30
 3     Mirri-Maz-Duur  female        7.30
 4              Jhogo    male        1.00)

In [6]:
# add edge labels
# edge_df.loc[:, 'label'] = edge_df.loc[:, 'relation'].astype(str)
Jaal(edge_df, node_df).plot(directed=True)

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jan/2022 09:49:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:49:41] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1641913912.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:49:41] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1641913912.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:49:41] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v0_13_1m1642154212.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:49:41] "GET /_dash-component-suites/visdcc/bundle.v0_0_40m1641913988.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:49:41] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1641913911.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:49:41] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_0_0m1641913912.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:49:41] "GET /

No trigger


In [7]:
# init Jaal and run server
Jaal(edge_df, node_df).plot(vis_opts={'height': '600px', # change height
                                      'interaction':{'hover': True}, # turn on-off the hover 
                                      'physics':{'stabilization':{'iterations': 100}}}) # define the convergence iteration of network

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jan/2022 09:50:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:50:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:50:16] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:50:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:50:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:50:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:50:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 09:50:16] "POST /_dash-update-component HTTP/1.1" 200 -


No trigger
